In [2]:
import requests as requests
import json
baseUrl = 'https://cloud.iexapis.com/v1'
apikey = 'pk_8a31fe45ab0c48c785d7f10d0021b0e9'
# baseUrl = 'https://sandbox.iexapis.com/v1'
# apikey = 'Tpk_0d3659b6498c4c3585a7d2ee190cbfc7'
from elasticsearch import Elasticsearch
es = Elasticsearch()
from pymongo import MongoClient
client = MongoClient()
db = client['test']
import math

In [3]:
def getSymbols(exchange):
#     exchange = 'nys' # nasdaq = nas; nyse = NYS
    response = requests.get(
        baseUrl + f'/ref-data/exchange/{exchange}/symbols',
        params= {
            'token': apikey,
            'format': 'json'
            }
        )
    symbols = json.loads(response.content)
    stockSymbols = list(filter(lambda x : x['type'] == 'cs', symbols))
    print(len(stockSymbols))
    return stockSymbols

In [4]:
def setSymbols(symbols):
    for stock in list(filter(lambda x : x['type'] == 'cs', symbols)):
        es.index(index="symbols", id=stock['symbol'].lower(), doc_type='_doc', body=stock)
        stock['_id'] = stock['symbol'].lower()
        db.symbols.update_one({'_id': stock['_id']}, {'$set': stock}, upsert=True)

In [5]:
def getExchanges():
    response = requests.get(
    baseUrl + f'/ref-data/exchanges',
    params= {
        'token': apikey,
        'format': 'json'
        }
    )
    exchanges = response.json()
    return exchanges

In [6]:
def getCompanyList(symbols):
    docs = {}
    for i in range(math.ceil(len(list(filter(lambda x : x['type'] == 'cs', symbols)))/100)):
        symbolStr = (','.join(map(lambda x: x['symbol'], list(filter(lambda x : x['type'] == 'cs', symbols))[(i)*100:(i+1) * 100])))
        response = requests.get(
        baseUrl + f'/stock/market/batch?symbols={symbolStr}&types=company',
        params= {
            'token': apikey,
            'format': 'json'
            }
        )
        newDocs = response.json()
        docs = {**docs, **newDocs}
    return docs

In [7]:
def setCompanyList(docs):
    for key, value in docs.items():
        try:
            print('----key----')
            print(key)
            doc = value['company']
            es.index(index="company", id=doc['symbol'].lower(), doc_type='_doc', body=doc)
            doc['_id'] = doc['symbol'].lower()
            db.company.update_one({'_id': doc['_id']}, {'$set': doc}, upsert=True)
        except Exception as e:
            print('------error-------')
            print(e)
            print(key)
            print(value)
            continue

In [ ]:
exchanges = getExchanges()
for exchange in exchanges:
    # symbols
    symbols = getSymbols(exchange['exchange'])
    setSymbols(symbols)
    # company
    docs = getCompanyList(symbols)
    setCompanyList(docs)

In [7]:
symbols = getSymbols('nas')

2582


In [8]:
docs = getCompanyList(symbols)

In [9]:
setCompanyList(docs)

----key----
AAL
----key----
AAME
----key----
AAOI
----key----
AAON
----key----
AAPL
----key----
AAWW
----key----
AAXN
----key----
ABCB
----key----
ABDC
----key----
ABEO
----key----
ABIL
----key----
ABIO
----key----
ABMD
----key----
ABTX
----key----
ABUS
----key----
ACAD
----key----
ACAM
----key----
ACBI
----key----
ACER
----key----
ACGL
----key----
ACHC
----key----
ACHN
----key----
ACHV
----key----
ACIA
----key----
ACIU
----key----
ACIW
----key----
ACLS
----key----
ACMR
----key----
ACNB
----key----
ACOR
----key----
ACRS
----key----
ACRX
----key----
ACST
----key----
ACTG
----key----
ACTT
----key----
ADBE
----key----
ADES
----key----
ADI
----key----
ADIL
----key----
ADMA
----key----
ADMP
----key----
ADMS
----key----
ADP
----key----
ADPT
----key----
ADRO
----key----
ADSK
----key----
ADTN
----key----
ADUS
----key----
ADVM
----key----
ADXS
----key----
AEGN
----key----
AEHR
----key----
AEIS
----key----
AEMD
----key----
AERI
----key----
AESE
----key----
AEY
----key----
AEYE
----key----
AEZS
-

----key----
CLDB
----key----
CLDC
----key----
CLDX
----key----
CLFD
----key----
CLIR
----key----
CLMT
----key----
CLNE
----key----
CLPS
----key----
CLRB
----key----
CLRO
----key----
CLSD
----key----
CLSN
----key----
CLUB
----key----
CLVS
----key----
CLWT
----key----
CLXT
----key----
CMBM
----key----
CMCO
----key----
CMCSA
----key----
CMCT
----key----
CME
----key----
CMEX
----key----
CMLS
----key----
CMPR
----key----
CMRX
----key----
CMTL
----key----
CNAT
----key----
CNBKA
----key----
CNCE
----key----
CNET
----key----
CNFR
----key----
CNMD
----key----
CNOB
----key----
CNSL
----key----
CNST
----key----
CNTY
----key----
CNXN
----key----
COCP
----key----
CODA
----key----
CODX
----key----
COHR
----key----
COHU
----key----
COKE
----key----
COLB
----key----
COLL
----key----
COLM
----key----
COMM
----key----
CONE
----key----
CONN
----key----
COOP
----key----
CORE
----key----
CORT
----key----
CORV
----key----
COST
----key----
COUP
----key----
COWN
----key----
CPAH
----key----
CPHC
----key----
C

----key----
GMLP
----key----
GNCA
----key----
GNLN
----key----
GNMK
----key----
GNMX
----key----
GNPX
----key----
GNTX
----key----
GNTY
----key----
GNUS
----key----
GO
----key----
GOGL
----key----
GOGO
----key----
GOOD
----key----
GOOG
----key----
GOOGL
----key----
GOSS
----key----
GPAQ
----key----
GPOR
----key----
GPP
----key----
GPRE
----key----
GPRO
----key----
GRBK
----key----
GRIF
----key----
GRIN
----key----
GRMN
----key----
GRNQ
----key----
GROW
----key----
GRPN
----key----
GRSH
----key----
GRTS
----key----
GSBC
----key----
GSHD
----key----
GSIT
----key----
GSKY
----key----
GSM
----key----
GT
----key----
GTHX
----key----
GTIM
----key----
GTLS
----key----
GTYH
----key----
GURE
----key----
GVP
----key----
GWGH
----key----
GWRS
----key----
GXGX
----key----
GYRO
----key----
HA
----key----
HABT
----key----
HAFC
----key----
HAIN
----key----
HAIR
----key----
HALL
----key----
HALO
----key----
HARP
----key----
HAS
----key----
HAYN
----key----
HBAN
----key----
HBCP
----key----
HBIO
----ke

----key----
MMSI
----key----
MMYT
----key----
MNCL
----key----
MNDO
----key----
MNKD
----key----
MNLO
----key----
MNOV
----key----
MNRO
----key----
MNSB
----key----
MNST
----key----
MNTA
----key----
MNTX
----key----
MOBL
----key----
MOFG
----key----
MOGO
----key----
MORF
----key----
MORN
----key----
MOSY
----key----
MOTS
----key----
MOXC
----key----
MPAA
----key----
MPB
----key----
MPVD
----key----
MPWR
----key----
MRAM
----key----
MRBK
----key----
MRCC
----key----
MRCY
----key----
MRIN
----key----
MRKR
----key----
MRLN
----key----
MRNA
----key----
MRNS
----key----
MRSN
----key----
MRTN
----key----
MRTX
----key----
MRUS
----key----
MRVL
----key----
MSBF
----key----
MSBI
----key----
MSEX
----key----
MSFT
----key----
MSON
----key----
MSTR
----key----
MSVB
----key----
MTBC
----key----
MTC
----key----
MTCH
----key----
MTEM
----key----
MTEX
----key----
MTRX
----key----
MTSC
----key----
MTSI
----key----
MTSL
----key----
MU
----key----
MUDS
----key----
MVBF
----key----
MVIS
----key----
MWK
--

----key----
RMTI
----key----
RNDB
----key----
RNET
----key----
RNST
----key----
RNWK
----key----
ROAD
----key----
ROCK
----key----
ROIC
----key----
ROKU
----key----
ROLL
----key----
ROSE
----key----
ROST
----key----
RP
----key----
RPAY
----key----
RPD
----key----
RRBI
----key----
RRGB
----key----
RRR
----key----
RTIX
----key----
RTLR
----key----
RTRX
----key----
RTTR
----key----
RUBY
----key----
RUN
----key----
RUSHA
----key----
RUSHB
----key----
RUTH
----key----
RVEN
----key----
RVLT
----key----
RVNC
----key----
RVSB
----key----
RWLK
----key----
RYTM
----key----
SABR
----key----
SAEX
----key----
SAFM
----key----
SAFT
----key----
SAGE
----key----
SAIA
----key----
SAL
----key----
SALM
----key----
SAMA
----key----
SAMG
----key----
SANM
----key----
SANW
----key----
SASR
----key----
SATS
----key----
SAUC
----key----
SAVA
----key----
SBAC
----key----
SBBP
----key----
SBBX
----key----
SBCF
----key----
SBFG
----key----
SBGI
----key----
SBLK
----key----
SBNY
----key----
SBPH
----key----
SBRA
-

----key----
WATT
----key----
WBA
----key----
WDAY
----key----
WDC
----key----
WDFC
----key----
WEBK
----key----
WEN
----key----
WERN
----key----
WETF
----key----
WEYS
----key----
WHF
----key----
WHLM
----key----
WHLR
----key----
WIFI
----key----
WILC
----key----
WINA
----key----
WING
----key----
WINS
----key----
WIRE
----key----
WISA
----key----
WIX
----key----
WKHS
----key----
WLDN
----key----
WLFC
----key----
WLTW
----key----
WMGI
----key----
WNEB
----key----
WORX
----key----
WPRT
----key----
WRLD
----key----
WRLS
----key----
WRTC
----key----
WSBC
----key----
WSBF
----key----
WSC
----key----
WSFS
----key----
WSTG
----key----
WSTL
----key----
WTBA
----key----
WTER
----key----
WTFC
----key----
WTRE
----key----
WTRH
----key----
WVE
----key----
WVFC
----key----
WVVI
----key----
WW
----key----
WWD
----key----
WWR
----key----
WYNN
----key----
XAIR
----key----
XBIO
----key----
XBIT
----key----
XCUR
----key----
XEL
----key----
XELA
----key----
XELB
----key----
XENE
----key----
XENT
----key--

In [8]:
exchanges = getExchanges()

In [9]:
exchanges

[{'exchange': 'ADS',
  'region': 'AE',
  'description': 'Abu Dhabi Securities Exchange',
  'mic': 'XADS',
  'exchangeSuffix': '-DH'},
 {'exchange': 'TAE',
  'region': 'IL',
  'description': 'Tel Aviv Stock Exchange',
  'mic': 'XTAE',
  'exchangeSuffix': '-IT'},
 {'exchange': 'BOM',
  'region': 'IN',
  'description': 'BSE Ltd.',
  'mic': 'XBOM',
  'exchangeSuffix': '-IB'},
 {'exchange': 'KRX',
  'region': 'KR',
  'description': 'Korea Exchange',
  'mic': 'XKRX',
  'exchangeSuffix': '-KP'},
 {'exchange': 'BRU',
  'region': 'BE',
  'description': 'Euronext Brussels',
  'mic': 'XBRU',
  'exchangeSuffix': '-BB'},
 {'exchange': 'ETR',
  'region': 'DE',
  'description': 'XETRA',
  'mic': 'XETR',
  'exchangeSuffix': '-GY'},
 {'exchange': 'PAR',
  'region': 'FR',
  'description': 'Euronext Paris',
  'mic': 'XPAR',
  'exchangeSuffix': '-FP'},
 {'exchange': 'LON',
  'region': 'GB',
  'description': 'London Stock Exchange',
  'mic': 'XLON',
  'exchangeSuffix': '-LN'},
 {'exchange': 'DUB',
  'regio

In [149]:
symbols = list(db['symbols'].find({'exchange': 'BOM'}))

In [150]:
len(symbols)

4701

In [151]:
symbols

[{'_id': '500002-ib',
  'currency': 'INR',
  'date': '2019-09-06',
  'exchange': 'BOM',
  'iexId': 'IEX_5747504E444A2D52',
  'isEnabled': True,
  'name': 'ABB India Limited',
  'region': 'IN',
  'symbol': '500002-IB',
  'type': 'cs'},
 {'_id': '500003-ib',
  'currency': 'INR',
  'date': '2019-09-06',
  'exchange': 'BOM',
  'iexId': 'IEX_4631483347582D52',
  'isEnabled': True,
  'name': 'Aegis Logistics Limited',
  'region': 'IN',
  'symbol': '500003-IB',
  'type': 'cs'},
 {'_id': '500008-ib',
  'currency': 'INR',
  'date': '2019-09-06',
  'exchange': 'BOM',
  'iexId': 'IEX_43524C544C502D52',
  'isEnabled': True,
  'name': 'Amara Raja Batteries Limited',
  'region': 'IN',
  'symbol': '500008-IB',
  'type': 'cs'},
 {'_id': '500009-ib',
  'currency': 'INR',
  'date': '2019-09-06',
  'exchange': 'BOM',
  'iexId': 'IEX_5751534650572D52',
  'isEnabled': True,
  'name': 'Ambalal Sarabhai Enterprises Limited',
  'region': 'IN',
  'symbol': '500009-IB',
  'type': 'cs'},
 {'_id': '500010-ib',
  

In [147]:
for stock in symbols:
    suffix =  [x for x in exchanges if x['exchange'] == stock['exchange']][0]['suffix']
    stock['yahooSymbol'] = stock['symbol'].split('-')[0] + (f'.{suffix}' if len(suffix) else '')
    print(stock['yahooSymbol'])

AALB.as
ACCEL.as
ACOMO.as
AD.as
ADYEN.as
AGDX.as
AGN.as
AJAX.as
AKZA.as
ALFEN.as
ALX.as
AMG.as
AND.as
APAM.as
ARCAD.as
ASM.as
ASML.as
ASN5.as
ASRNL.as
ATC.as
ATCB.as
ATRS.as
AVTX.as
AXS.as
BAMA.as
BAMNB.as
BBED.as
BESI.as
BEVER.as
BFIT.as
BINCK.as
BOKA.as
BOLS.as
BRILL.as
BRNL.as
BSGR.as
CCEP.as
CLB.as
CRBN.as
CTAC.as
CURE.as
DGB.as
DPA.as
DSC1.as
DSM.as
EAS2P.as
ENVI.as
ESP.as
FFARM.as
FLOW.as
FNG.as
GLPG.as
GTO.as
GVNV.as
HAL.as
HDG.as
HEIA.as
HEIJM.as
HEIO.as
HOLCO.as
HYDRA.as
ICT.as
IEX.as
IMCD.as
INGA.as
INTER.as
KARD.as
KDS.as
KENDR.as
KPN.as
KVW.as
LI.as
LIGHT.as
LVIDE.as
MAKIT.as
MAREL.as
MORE.as
MSHIA.as
MT.as
MTS.as
NEDAP.as
NEDSE.as
NEWAY.as
NIBC.as
NN.as
NOVI.as
NRP.as
NSE.as
NSI.as
OCI.as
ORANW.as
ORDI.as
PHARM.as
PHIA.as
PNL.as
PORF.as
PPLAA.as
PPLAB.as
PRX.as
RAND.as
RDSA.as
RDSB.as
REINA.as
REN.as
RICOA.as
ROOD.as
SBMO.as
SIFG.as
SLIGR.as
SNOW.as
STRN.as
TBIRD.as
TIE.as
TKWY.as
TOM2.as
UNA.as
UNIS.as
VALUE.as
VASTN.as
VEON.as
VPK.as
VVY.as
WDP.as
WES.as
WHA.as
WKL.as
WM

In [103]:
exchanges = [
    {'exchange': 'NAS', 'suffix': ''},
    {'exchange': 'NYS', 'suffix': ''},
    {'exchange': 'ASE', 'suffix': ''},
    {'exchange': 'LON', 'suffix': 'l'},
    {'exchange': 'PAR', 'suffix': 'pa'},
    {'exchange': 'AMS', 'suffix': 'as'},
    {'exchange': 'BRU', 'suffix': 'br'},
    {'exchange': 'LIS', 'suffix': 'ls'},
    {'exchange': 'TSE', 'suffix': 'to'},
    {'exchange': 'TSX', 'suffix': 'v'},
    {'exchange': 'MEX', 'suffix': 'mx'},
    {'exchange': 'ETR', 'suffix': 'de'},
    {'exchange': 'KRX', 'suffix': ['kq', 'ks']},
    {'exchange': 'BOM', 'suffix': 'bo'},
    {'exchange': 'TAE', 'suffix': 'ta'}
]

In [102]:
symbols

[{'_id': 'abnic-dh',
  'currency': 'AED',
  'date': '2019-09-06',
  'exchange': 'ADS',
  'iexId': 'IEX_425744344E312D52',
  'isEnabled': True,
  'name': 'Al Buhaira National Insurance Co.',
  'region': 'AE',
  'symbol': 'ABNIC-DH',
  'type': 'cs'},
 {'_id': 'adaviation-dh',
  'currency': 'AED',
  'date': '2019-09-06',
  'exchange': 'ADS',
  'iexId': 'IEX_544D584C4B522D52',
  'isEnabled': True,
  'name': 'Abu Dhabi Aviation',
  'region': 'AE',
  'symbol': 'ADAVIATION-DH',
  'type': 'cs'},
 {'_id': 'adcb-dh',
  'currency': 'AED',
  'date': '2019-09-06',
  'exchange': 'ADS',
  'iexId': 'IEX_5644575735392D52',
  'isEnabled': True,
  'name': 'Abu Dhabi Commercial Bank',
  'region': 'AE',
  'symbol': 'ADCB-DH',
  'type': 'cs'},
 {'_id': 'adib-dh',
  'currency': 'AED',
  'date': '2019-09-06',
  'exchange': 'ADS',
  'iexId': 'IEX_5151325335352D52',
  'isEnabled': True,
  'name': 'Abu Dhabi Islamic Bank',
  'region': 'AE',
  'symbol': 'ADIB-DH',
  'type': 'cs'},
 {'_id': 'adnh-dh',
  'currency'

In [148]:
[x['exchange'] for x in exchanges]

['NAS',
 'NYS',
 'ASE',
 'LON',
 'PAR',
 'AMS',
 'BRU',
 'LIS',
 'TSE',
 'TSX',
 'MEX',
 'ETR',
 'KRX',
 'BOM',
 'TAE']